# **GROUP 10**

**Group Members**  
Saaumitra Raaj (220928), Swarnim Verma (231071), Abdul Ahad Zareef (230030)

**IMPORTING THE NECESSARY LIBRARIES**

In [1]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from vtk import *
from vtkmodules.util import numpy_support

**LOADING THE DATASET**

CHANGE THE PATH OF THE DATASET AS PER YOUR SYSTEM


In [2]:
# Load the dataset
reader = vtkXMLImageDataReader()
reader.SetFileName("/Users/saaumitraraaj/Desktop/CS661/Assignment2/mixture.vti") # change the path of your dataset
reader.Update()
vti_data = reader.GetOutput()

GRID & SCALAR DATA EXTRACTION

In [3]:
point_data = vti_data.GetPointData()
scalar_name = point_data.GetArrayName(0)
vtk_array = point_data.GetArray(scalar_name)
flat_val = numpy_support.vtk_to_numpy(vtk_array)

# Grid and scalar data 
dims = vti_data.GetDimensions()
volume_data = flat_val.reshape(dims, order='F')

**ISOSURFACE AND HISTOGRAM FUNCTION**

In [4]:
# Extract scalar field
x = np.arange(0, dims[0])
y = np.arange(0, dims[1])
z = np.arange(0, dims[2])
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

flat_x = X.flatten()
flat_y = Y.flatten()
flat_z = Z.flatten()
flat_val = volume_data.flatten()

data_min = float(np.min(flat_val))
data_max = float(np.max(flat_val))

# Iso-surface function
def create_isosurface(isovalue):
    fig = go.Figure()
    fig.add_trace(go.Isosurface(
        x=flat_x, y=flat_y, z=flat_z,
        value=flat_val,
        isomin=isovalue,
        isomax=isovalue,
        surface_count=1,
        colorscale='plasma',
        cmin=data_min,
        cmax=data_max,
        caps=dict(x_show=False, y_show=False, z_show=False),
        showscale=False  
    ))
    fig.update_layout(
        title=f"Isosurface at isovalue = {isovalue:.2f}",
        width=500, height=500,
        margin=dict(l=0, r=0, t=30),
        scene=dict(
            xaxis=dict(title='x', showticklabels=False),
            yaxis=dict(title='y', showticklabels=False),
            zaxis=dict(title='z', showticklabels=False)
        )
    )
    return fig

# Histogram function
def create_histogram(isovalue=None):
    if isovalue is None:
        data = flat_val
    else:
        mask = np.abs(flat_val - isovalue) <= 0.25
        data = flat_val[mask]

    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=data,
        nbinsx=50,
        marker_color='cornflowerblue' # almost matching with the colour depicted in the samples :)
    ))
    fig.update_layout(
        xaxis_title="Vortex scalar values",
        yaxis_title="Frequency",
        yaxis=dict(tickformat="~s"),  # 5k instead of 5000
        width=500,
        height=500
    )
    return fig

WIDGETS INITIALIZATION - SLIDER AND RESET BUTTON

In [5]:
# Initialising the widgets
slider = widgets.FloatSlider(
    value=0.0,
    min=data_min,
    max=data_max,
    step=0.01,
    description='Isovalue:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

reset_button = widgets.Button(description="Reset", button_style='info')

# Using FigureWidget instead of Figure to avoid re-rendering
isosurface_widget = go.FigureWidget(create_isosurface(0.0))
histogram_widget = go.FigureWidget(create_histogram())

**FUNCTION TO UPDATE THE PLOTS AS PER THE ISOVALUE**

In [6]:
def update_plots(isovalue, hist=True):
    isosurface_widget.data[0].isomin = isovalue
    isosurface_widget.data[0].isomax = isovalue
    isosurface_widget.layout.title.text = f"Isosurface at isovalue = {isovalue:.2f}"
    
    if hist:
        mask = np.abs(flat_val - isovalue) <= 0.25
        histogram_widget.data[0].x = flat_val[mask]
    else:
        histogram_widget.data[0].x = flat_val

**RESET FUNCTION** ->  RESETS TO ISOVALUE = 0.0

In [7]:
def reset(event=None):
    slider.value = 0.0
    update_plots(0.0, hist=False)

slider.observe(lambda change: update_plots(change['new']), names='value')
reset_button.on_click(reset)

**DISPLAY THE FINAL LAYOUT**

In [8]:
layout = widgets.VBox([
    widgets.HBox([slider, reset_button]),
    widgets.HBox([isosurface_widget, histogram_widget])
])
display(layout)

